# Sequence Generation

In this exercise, you will design an RNN to generate baby names! You will design an RNN to learn to predict the next letter of a name given the preceding letters. This is a character-level RNN rather than a word-level RNN.

This idea comes from this excellent blog post: http://karpathy.github.io/2015/05/21/rnn-effectiveness/

In [ ]:
%matplotlib inline

import numpy as np
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import np_utils
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Embedding
from tensorflow.keras.layers import LSTM, SimpleRNN, GRU

## Training Data

The training data we will use comes from this corpus:
http://www.cs.cmu.edu/afs/cs/project/ai-repository/ai/areas/nlp/corpora/names/

Take a look at the training data in `data/names.txt`, which includes both boy and girl names. Below we load the file and convert it to all lower-case for simplicity.

Note that we also add a special "end" character (in this case a period) to allow the model to learn to predict the end of a name.

In [ ]:
with open('../data/names.txt') as f:
    names = f.readlines()
    names = [name.lower().strip() + '.' for name in names]

print('Loaded %d names' % len(names))

In [ ]:
names[:10]

We need to count all of the characters in our "vocabulary" and build a dictionary that translates between the character and its assigned index (and vice versa).

In [ ]:
chars = set()
for name in names:
    chars.update(name)
vocab_size = len(chars)
print('Vocabulary size:', vocab_size)

char_inds = dict((c, i) for i, c in enumerate(chars))
inds_char = dict((i, c) for i, c in enumerate(chars))

In [ ]:
char_inds

#### Exercise 1 - translate chars to indexes

Most of the work of preparing the data is taken care of, but it is important to know the steps because they will be needed anytime you want to train an RNN. Use the dictionary created above to translate each example in `names` to its number format in `int_names`.

In [ ]:
# Translate names to their number format in int_names

The `create_matrix_from_sequences` will take the examples and create training data by cutting up names into input sequence of length `maxlen` and training labels, which are the following character. Make sure you understand this procedure because it is what will actually go into the network!

In [ ]:
def create_matrix_from_sequences(int_names, maxlen, step=1):
    name_parts = []
    next_chars = []
    for name in int_names:
        for i in range(0, len(name) - maxlen, step):
            name_parts.append(name[i: i + maxlen])
            next_chars.append(name[i + maxlen])

    return name_parts, next_chars

maxlen = 3
name_parts, next_chars = create_matrix_from_sequences(int_names, maxlen)
print('Created %d name segments' % len(name_parts))

In [ ]:
X_train = sequence.pad_sequences(name_parts, maxlen=maxlen)
y_train = to_categorical(next_chars, vocab_size)

In [ ]:
X_train.shape

In [ ]:
X_train[:5]

#### Exercise 2 - design a model

Design your model below. Like before, you will need to set up the embedding layer, the recurrent layer, a dense connection and a softmax to predict the next character.

Fit the model by running at least 10 epochs. Later you will generate names with the model. Getting around 30% accuracy will usually result in decent generations. What is the accuracy you would expect for random guessing?

In [ ]:
model.fit(X_train, y_train, batch_size=32, epochs=10, verbose=1)

## Sampling from the model

We can sample the model by feeding in a few letters and using the model's prediction for the next letter. Then we feed the model's prediction back in to get the next letter, etc.

The `sample` function is a helper to allow you to adjust the diversity of the samples. You can read more [here](https://en.wikipedia.org/wiki/Softmax_function#Reinforcement_learning).

Read the `gen_name` function to understand how the model is sampled.

In [ ]:
def sample(p, diversity=1.0):
    p1 = np.asarray(p).astype('float64')
    p1 = np.log(p1) / diversity
    e_p1 = np.exp(p1)
    s = np.sum(e_p1)
    p1 = e_p1 / s
    return np.argmax(np.random.multinomial(1, p1, 1))


def gen_name(seed, length=1, diversity=1.0, maxlen=3):
    """
    seed - the start of the name to sample
    length - the number of letters to sample; if None then samples
        are generated until the model generates a '.' character
    diversity - a knob to increase or decrease the randomness of the
        samples; higher = more random, lower = closer to the model's
        prediction
    maxlen - the size of the model's input
    """
    
    # Prepare input array
    x = np.zeros((1, maxlen), dtype=int)

    # Generate samples
    out = seed
    while length is None or len(out) < len(seed) + length:

        # Add the last chars so far for the next input
        for i, c in enumerate(out[-maxlen:]):
            x[0, i] = char_inds[c]
        
        # Get softmax for next character
        preds = model.predict(x, verbose=0)[0]
        
        # Sample the network output with diversity
        c = sample(preds, diversity)
        
        # Choose to end if the model generated an end token
        if c == char_inds['.']:
            if length is None:
                return out
            else:
                continue

        # Build up output
        out += inds_char[c]
        
    return out

#### Exercise 3 - sample the model

Use the `gen_name` function above to sample some names from your model.

1. Try generating a few characters by setting the `length` argument.
2. Try different diversities. Start with 1.0 and vary it up and down.
3. Try using `length=None`, allowing the model to choose when to end a name.
4. What happens when `length=None` and the diversity is high? How do samples change in this case staring from beginning to end? Why do you think this is?
5. With `length=None` and a "good" diversity, can you tell if the model has learned a repertoire of "endings"? What are some of them? 
6. Find some good names. What are you favorites? :D

#### Exercise 4 - retrain

Now that you have seen some samples, go back up and redefine your model to "erase" it. Don't train it again yet. You can sample again to compare the quality of the samples before the model is trained.

Experiment with the hidden layer size, the maxlen, the number of epochs, etc. Do you observe any differences in the sample behavior?

Not all changes will make an observable impact, but do experiments to see what you can discover.

Finally, take a look at [this Colab notebook](https://colab.research.google.com/github/tensorflow/tpu/blob/master/tools/colab/shakespeare_with_tpu_and_keras.ipynb) for a similar example with Shakespeare text